# Multi-Client Example

## Introduction

This example shows how to connect multiple `BeamNGpy` instances to the simulator and have them control different vehicles. For demonstration purposes, both instances are housed in the same Python process, but the same example could be used from an entirely different process or machine as well.

## Scenario

The scenario will be a rather simple one: there are two vehicles on the `smallgrid` map (an infinite flat plane) and one vehicle will randomly drive around, controlled by client A, with another vehicle, controlled by client B, mimicking A's behavior.

## Setup

Contrary to other examples, we will be using two instances of the `BeamNGpy` class representing two clients A and B. In our case, client A will create the scenario containing both vehicles, but client B will later connect and control one of the vehicles while A controls the other. The classes involved in this are:

* `BeamNGpy`: Two instances of this will be used to implement client A and B
* `Scenario`: Client A will create a scenario for both clients to play in
* `Vehicle`: Used to specify and control vehicles involved in the scenario
* `Electrics`: A sensor used to inspect interal systems of the vehicle

The code starts with importing the respective classes:

In [1]:
import random

import numpy as np

from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics

Now we set up our first client who will create the scenario for both clients. Client A will also launch the simulator.

In [2]:
client_a = BeamNGpy('localhost', 64256)
client_a.open()

With the simulator running, we can start setting up our scenario. It will contain two vehicles, the one controlled by Client A being placed in front of the one later controlled by client B.

In [3]:
scenario = Scenario('smallgrid', 'tag')
av_a = Vehicle('vehicleA', model='etk800')
av_b = Vehicle('vehicleB', model='etk800')
scenario.add_vehicle(av_a, pos=(0, -10, 0))
scenario.add_vehicle(av_b)
scenario.make(client_a)

## Running

The scenario is now made, meaning the required files have been generated and can be loaded in the simulator:

In [4]:
client_a.load_scenario(scenario)
client_a.start_scenario()

Now we will set up our second client and connect it to the running simulator. The client will first connect, then query the running scenario, and retrieve currently active vehicles. They will then find the vehicle meant for Client B and connect to it. Note that `client_b` is opened with two flags `launch=False` and `deploy=False` meaning it will not launch its own BeamNG.tech process and not deploy the Lua files necessary for communication as Client A has already done so.

In [5]:
client_b = BeamNGpy('localhost', 64256)
client_b.open(launch=False, deploy=False)
running_scenario = client_b.get_current_scenario()
print(running_scenario.name)
active_vehicles = client_b.get_current_vehicles()
bv_a = active_vehicles['vehicleA']
bv_b = active_vehicles['vehicleB']
# B attaches their own sensor to get the current controls of A
bv_a.attach_sensor('electrics', Electrics())
bv_a.connect(client_b)
bv_b.connect(client_b)

tag


Two clients are now connected to the running simulation and both vehicles. What follows is the main loop of the scenario, where Client A sends random steering inputs to their vehicle and Client B checks how A's vehicle is driving using the electrics sensor and sends the same inputs to their vehicle.

In [6]:
# Focus simulator on second vehicle because it's the more interesting one to see
client_a.switch_vehicle(av_b)

for _ in range(120):
    # Client A sending control inputs to their vehicle connection
    av_a.control(throttle=random.random(), steering=(random.random() * 2 - 1))
    
    # Client B updating the electrics sensor of A's vehicle
    bv_a.poll_sensors()
    
    throttle = bv_a.sensors['electrics'].data['throttle_input']
    steering = bv_a.sensors['electrics'].data['steering_input']
    print(throttle, steering)
    bv_b.control(throttle=throttle, steering=steering)
    
    # Client A now advancing the simulation 60 steps
    client_a.step(60)

client_b.close()
client_a.close()

0.17316560355877775 -0.08137247713993863
0.2245978039112846 -0.4669812479331782
0.6309506494197058 0.2990009632308111
0.8662440221625067 -0.8740858308898218
0.22348886211721652 -0.7906601926514055
0.28467718400361963 -0.557201380577362
0.9062191595767862 -0.1919533939659307
0.29767996666474217 0.1362717779570709
0.1194594550230087 0.8864501474166042
0.9660245397509041 0.7024685659545233
0.1918685051500344 0.13618006004285987
0.3873766034334287 0.23259978071938658
0.09708406341805664 -0.6515072093174388
0.500591338395343 -0.15141255143421536
0.021660823643308458 0.40209764032728046
0.30375545984087304 0.30091720413308726
0.48781119851984633 0.35782509734987167
0.1682884358807526 -0.3554365707400838
0.3095892955505989 0.2765067242028339
0.22299518082737402 0.03622262657787638
0.5967267475752369 -0.0016975713076148313
0.8557363317327791 -0.1202967102304329
0.5719057059545953 -0.6392819679388321
0.8139803389894057 -0.5460076795431187
0.20880170156772415 0.36301691372873035
0.96740760401940

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host